<a href="https://colab.research.google.com/github/mahmoudmagdyhassan/NLP-and-CV/blob/main/car_speed_and_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/AhmedIbrahimai/Counting-people-in-a-marathon-using-YOLOv8.git
!pip install ultralytics
!pip install cvzone


In [ ]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import*
import cvzone
model=YOLO('yolov8s.pt')

In [5]:
from google.colab.patches import cv2_imshow
import cv2
import cvzone
import time

In [ ]:
# Define the video capture
cap = cv2.VideoCapture('/content/drive/MyDrive/CNN/veh2.mp4')

# Read the class list from the file
my_file = open("/content/coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")

# Tracker instance
tracker = Tracker()

# Line coordinates
cy1 = 322
cy2 = 368
offset = 6

# Dictionaries for tracking vehicles going up and down
vh_down = {}
counter = []
vh_up = {}
counter1 = []

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # You can change the codec as needed
output_video = cv2.VideoWriter('/content/drive/MyDrive/CNN/output_video2.avi', fourcc, 20.0, (1020, 500))

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Process every third frame
    count += 1
    if count % 3 != 0:
        continue

    # Resize the frame
    frame = cv2.resize(frame, (1020, 500))

    # Model prediction (assuming you have a model named 'model' defined elsewhere in your code)
    results = model.predict(frame)
    boxes_model = results[0].boxes.xyxy.cpu()
    class_labels_model = results[0].boxes.cls.cpu().tolist()

    # Bounding box list
    bbox_list = []

    # Process results and draw bounding boxes for specific classes
    for box, label in zip(boxes_model, class_labels_model):
        x, y, x_max, y_max = map(int, box[:4])

        # Add labels for specific classes
        if label in [1, 2, 3, 5, 7]:  # Classes for bicycle, car, motorcycle, bus, truck
            object_label = class_list[int(label)]
            bbox_list.append([x, y, x_max, y_max])

            # Draw bounding box with label
            cv2.rectangle(frame, (x, y), (x_max, y_max), (0, 0, 255), 2)
            cv2.putText(frame, object_label, (x, y - 10), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)

    # Update tracker and process vehicle movement
    bbox_id = tracker.update(bbox_list)
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        # Process vehicles going down
        if cy1 < (cy + offset) and cy1 > (cy - offset):
            vh_down[id] = time.time()
        if id in vh_down:
            if cy2 < (cy + offset) and cy2 > (cy - offset):
                elapsed_time = time.time() - vh_down[id]
                if counter.count(id) == 0:
                    counter.append(id)
                    distance = 10  # meters
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

        # Process vehicles going up
        if cy2 < (cy + offset) and cy2 > (cy - offset):
            vh_up[id] = time.time()
        if id in vh_up:
            if cy1 < (cy + offset) and cy1 > (cy - offset):
                elapsed1_time = time.time() - vh_up[id]
                if counter1.count(id) == 0:
                    counter1.append(id)
                    distance1 = 10  # meters
                    a_speed_ms1 = distance1 / elapsed1_time
                    a_speed_kh1 = a_speed_ms1 * 3.6
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh1)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    # Draw lines and display information
    cv2.line(frame, (274, cy1), (814, cy1), (255, 255, 255), 1)
    cv2.putText(frame, 'L1', (277, 320), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
    cv2.line(frame, (177, cy2), (927, cy2), (255, 255, 255), 1)
    cv2.putText(frame, 'L2', (182, 367), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    # Display vehicle count
    d = len(counter)
    u = len(counter1)
    cv2.putText(frame, 'goingdown: ' + str(d), (60, 90), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
    cv2.putText(frame, 'goingup: ' + str(u), (60, 130), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    # Show the frame
    cv2_imshow(frame)

    # Save the frame to the output video
    output_video.write(frame)

    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the VideoWriter and cleanup
output_video.release()
cap.release()
cv2.destroyAllWindows()
